In [2]:
!git config --global user.name "ek-Nelson"
!git config --global user.email "ksimonelson@gmail.com"

In [5]:
!git clone https://github.com/ek-Nelson/Waste_detection.git

Cloning into 'Waste_detection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import os
import json
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import torchvision.transforms as transforms
from torchvision.ops import box_iou

In [ ]:
import json

# Fonction pour charger les annotations
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        annotations = json.load(f)
    return annotations

# Chemin vers ton fichier d'annotations
annotations_file = "/content/drive/MyDrive/Colab Notebooks/Dataset/train/_annotations.createml.json"  # Modifie le chemin si nécessaire

# Charger les annotations
annotations = load_annotations(annotations_file)

# Extraire les noms des classes
def extract_labels(annotations):
    labels = set()  # Utilisation d'un ensemble pour éviter les doublons
    for annotation in annotations:
        for obj in annotation["annotations"]:
            labels.add(obj["label"])
    return sorted(labels)  # Retourne les labels triés pour plus de lisibilité

# Afficher les labels
labels = extract_labels(annotations)
print("Labels trouvés dans les annotations JSON :", labels)


Labels trouvés dans les annotations JSON : ['cardboard', 'glass', 'metal', 'plastic']


In [ ]:
!pip install ipdb

In [ ]:
import os
import json
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import torchvision.transforms as transforms
from torchvision.ops import box_iou
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Charger les annotations JSON
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        annotations = json.load(f)
    return annotations


# Dataset pour la détection d'objets
class ObjectDetectionDataset(Dataset):
    def __init__(self, annotations, images_dir, transform=None, label_map=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.transform = transform

        if label_map is None:
            self.label_map = self.build_label_map()
        else:
            self.label_map = label_map


    def build_label_map(self):
        labels = set()
        for annotation in self.annotations:
            for obj in annotation["annotations"]:
                labels.add(obj["label"])
        print("classes détectées dans le json: ", labels)
        return {label: idx for idx, label in enumerate(sorted(labels))}


    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        image_name = annotation["image"]
        image_path = os.path.join(self.images_dir, image_name)

        # Charger l'image
        image = cv2.imread(image_path)
        print(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Charger les bounding boxes et labels
        coords = []
        labels = []
        for obj in annotation["annotations"]:
            x, y = obj["coordinates"]["x"], obj["coordinates"]["y"]
            w, h = obj["coordinates"]["width"], obj["coordinates"]["height"]
            coords.append([x, y, w, h])
            labels.append(self.label_map[obj["label"]])

        coords = torch.tensor(coords, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        # Appliquer des transformations
        if self.transform:
            image = self.transform(image)

        return (image, {'coords': coords, 'labels': labels})

# Préparer le transformateur pour les images
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Resize((300, 300)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Charger les annotations et initialiser le DataLoader
annotations_file = "/content/drive/MyDrive/Colab Notebooks/Dataset/train/_annotations.createml.json"  # Remplacez par le chemin correct
images_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset/train"  # Remplacez par le dossier contenant les images

annotations = load_annotations(annotations_file)
label_map = {"plastic": 0, "cardboard": 1, "glass": 2, "metal": 3}
dataset = ObjectDetectionDataset(
    annotations,
    images_dir,
    transform=transform,
    label_map=label_map)

dataloader = DataLoader(dataset,
                        batch_size=4,
                        shuffle=True)

# Construire le modèle avec ResNet en backbone
class DetectionModel(nn.Module):
    def __init__(self, num_classes):
        super(DetectionModel, self).__init__()
        backbone = models.resnet50(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-2])  # Supprime les couches FC

        self.conv = nn.Conv2d(2048, 1024, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(1024, num_classes)  # Classification
        self.regressor = nn.Linear(1024, 4)  # Régression des bounding boxes

    def forward(self, x):
        features = self.feature_extractor(x)
        features = self.conv(features)
        pooled_features = self.pool(features)
        pooled_features = pooled_features.flatten(start_dim=1)


        class_logits = self.classifier(pooled_features)
        coords = self.regressor(pooled_features)
        return class_logits, coords

# Initialiser le modèle, la perte et l'optimiseur
num_classes = len(label_map)

model = DetectionModel(num_classes)

criterion_class = nn.CrossEntropyLoss()
criterion_coords = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Entraîner le modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):
    model.train()
    total_loss = 0
    for images, targets in dataloader:
        import ipdb; ipdb.set_trace()
        images = images.to(device)
        labels = torch.cat([t['labels'] for t in targets]).to(device)
        coords = torch.cat([t['coords'] for t in targets]).to(device)

        # Forward pass
        class_logits, predicted_coords = model(images)

        # Calcul des pertes
        print(class_logits.shape)
        num_classes = len(labels)
        print(num_classes)
        print(labels.shape)
        print(predicted_coords.shape)
        print(coords.shape)
        loss_class = criterion_class(class_logits, labels)
        loss_coords = criterion_coords(predicted_coords, coords)
        loss = loss_class + loss_coords

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


/content/drive/MyDrive/Colab Notebooks/Dataset/train/IMG_9139_27_11zon_jpg.rf.d8ff3dd7806cecb1f02945e1c07cf1a1.jpg
/content/drive/MyDrive/Colab Notebooks/Dataset/train/IMG_9276_15_11zon_jpg.rf.dd75e99645575ed39dc99d654990312b.jpg
/content/drive/MyDrive/Colab Notebooks/Dataset/train/IMG_9251_7_11zon_jpg.rf.3aad3bc330dec2f788111869603bc946.jpg
/content/drive/MyDrive/Colab Notebooks/Dataset/train/IMG_9334_78_11zon_jpg.rf.612873c2a3f917114b064649533bbbcd.jpg


RuntimeError: stack expects each tensor to be equal size, but got [1, 4] at entry 0 and [2, 4] at entry 1

In [ ]:
def draw_bounding_boxes(image, predicted_coords, labels, label_map):
    plt.imshow(image.permute(1, 2, 0).cpu().numpy())
    for coord, label in zip(predicted_coords, labels):
        x, y, w, h = coord
        x_min, y_min = x - w / 2, y - h / 2
        rect = patches.Rectangle(
            (x_min, y_min), w, h,
            linewidth=2, edgecolor='r', facecolor='none'
        )
        plt.gca().add_patch(rect)
        plt.text(x_min, y_min, list(label_map.keys())[label.item()], color='r', fontsize=10)
    plt.axis('off')
    plt.show()

# Inférence et visualisation
model.eval()
with torch.no_grad():
    for images, targets in dataloader:
        images = torch.stack(images).to(device)
        class_logits, predicted_coords = model(images)

        for i in range(len(images)):
            draw_bounding_boxes(images[i], predicted_coords[i].cpu(), torch.argmax(class_logits, dim=1).cpu(), label_map)
        break

In [ ]:
# load the dataset
train_dataset =

In [6]:
%cd /content/Waste_detection

/content/Waste_detection


In [11]:
!git add .
!git commit -m "first code commit that contents such errors mentionned yesterday"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [13]:
!git pull

Already up to date.


In [14]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
